# VEHICLE DETECTION PROJECT

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle
from scipy.ndimage.measurements import label
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from moviepy.editor import VideoFileClip

# Common attributes and functions
### Map with the color scales

In [ ]:
convColor = {'YUV': cv2.COLOR_RGB2YUV, 'YCrCb': cv2.COLOR_RGB2YCrCb, 
            'HSV': cv2.COLOR_RGB2HSV, 'LUV': cv2.COLOR_RGB2LUV,
            'HLS': cv2.COLOR_RGB2HLS, 'YUV': cv2.COLOR_RGB2YUV,
            'YCrCb': cv2.COLOR_RGB2YCrCb, 'LAB': cv2.COLOR_RGB2LAB}

### Function to return HOG features

In [ ]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# HOG Features Extraction and Train a Linear SVM Classifier
## HOG Features Extraction and Train-Test Examples Definition
### Function to extract HOG features from images

In [ ]:
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0, spatial_size=(32,32), hist_bins=32):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)

        # Convert to the selected color scale
        if cspace != 'RGB':
            feature_image = cv2.cvtColor(image, convColor[cspace])
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

### Load images files and split up time series vehicles images into train and test examples.
In the vehicle dataset, the GIT* folders contain time-serie data. To optimize de classifier I split the images to make sure train and test images are suffient different from one another.

In [ ]:
vehicles_train = []
vehicles_test = []
non_vehicles = []
files = glob.glob('./clf_images/vehicles/GTI_Far/*.png')
indx_test = int(len(files) * 0.2)
vehicles_train.append(files[:indx_test])
vehicles_test.append(files[indx_test:])
files = glob.glob('./clf_images/vehicles/GTI_Left/*.png')
indx_test = int(len(files) * 0.2)
vehicles_train.append(files[:indx_test])
vehicles_test.append(files[indx_test:])
files = glob.glob('./clf_images/vehicles/GTI_MiddleClose/*.png')
indx_test = int(len(files) * 0.2)
vehicles_train.append(files[:indx_test])
vehicles_test.append(files[indx_test:])
files = glob.glob('./clf_images/vehicles/GTI_Right/*.png')
indx_test = int(len(files) * 0.2)
vehicles_train.append(files[:indx_test])
vehicles_test.append(files[indx_test:])
files = glob.glob('./clf_images/vehicles/KITTI_extracted/*.png')
indx_test = int(len(files) * 0.2)
vehicles_train.append(files[:indx_test])
vehicles_test.append(files[indx_test:])
vehicles_train = np.concatenate(vehicles_train)
vehicles_test = np.concatenate(vehicles_test)
print('# vehicles_train {} - vehicles_test'.format(len(vehicles_train)), len(vehicles_test))

files = glob.glob('./clf_images/non-vehicles/GTI/*.png')
non_vehicles.append(files)
files = glob.glob('./clf_images/non-vehicles/Extras/*.png')
non_vehicles.append(files)
non_vehicles = np.concatenate(non_vehicles)
print('# non vehicles {}'.format(len(non_vehicles)))

### HOG features extraction parameters.

In [ ]:
colorspace = 'YUV' # Can be RGB, HSV, HLS, YUV, YCrCb
orient = 11
pix_per_cell = 16
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

### HOG features extraction from vehicles and non-vehicle images.
The extraction of HOG features of the vehicle images is done separately for the training examples and the test examples.

In [ ]:
t=time.time()
car_features_train = extract_features(vehicles_train, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
car_features_test = extract_features(vehicles_test, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

notcar_features = extract_features(non_vehicles, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

### Split up non-vehicles features into randomized training and test sets.
The vehicle features ara already split. The non-vehicle labels vector is also defined.

In [ ]:
rand_state = np.random.randint(0, 100)
X_nc_train, X_nc_test, y_nc_train, y_nc_test = train_test_split(
    notcar_features, np.zeros(len(notcar_features)), test_size=0.2, random_state=rand_state)

### Create an array stack of feature vectors, define the vehicles labels vectors and shuffle the train features and labels vectors.

In [ ]:
X_train = np.vstack((car_features_train, X_nc_train)).astype(np.float64)
X_test  = np.vstack((car_features_test, X_nc_test)).astype(np.float64)

y_train = np.hstack((np.ones(len(car_features_train)), y_nc_train))
y_test = np.hstack((np.ones(len(car_features_test)), y_nc_test))

X_train, y_train = shuffle(X_train, y_train)

## Trainnig and Parameter Tuning the Linear SVM Classifier
To tune the Linear SVM vehicle detection model, I used the `GridSearchCV` scikit-learn's parameter tuning algoritm.
### Tuning parameters

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']}
 ]

### Fit SVM Classifier

In [ ]:
print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

svr = SVC()
clf = GridSearchCV(svr,param_grid)
# Check the training time for the SVC

t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

### Access to the optimal parameter combination.

In [ ]:
best_params = clf.best_params_
print(best_params)

### Test accuracy of SVC

In [ ]:
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))

### Save the model

In [ ]:
model = {'svc': clf,
          'orient': orient,
          'colorspace': colorspace,
          'pix_per_cell': pix_per_cell,
          'cell_per_block': cell_per_block}
pickle.dump( model, open( "./models/hog_model.p", "wb" ))

# Vehicule Detection
### Load the model and extract HOG parameters

In [ ]:
dist_pickle = pickle.load( open("./models/hog_model.p", "rb" ) )

# get attributes of our svc object
svc = dist_pickle["svc"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
color_scale = dist_pickle['colorspace']

### Function to define the image patches in which to search for vehicles

In [ ]:
def get_image_patches(img, first, restart, car_history):
    h, w, _ = img.shape
    left = int(64 * 4.65) + 1
    right = w - int(64 * 4.65) + 1 
    patches = []
    # If it is the first video frame, an exhaustive search is made in the lower half of the frame.
    if first: 
        first_patches = [(1.0, (392, 480, 0, w), 1), (1.3, (392, 497, 0, w), 1), 
                         (1.6, (392, 546, 0, w), 1), (2.0, (392, 590, 0, w), 1), 
                        (2.5, (392, 633, 0, w), 1), (2.85, (392, 666, 0, w), 1)]
        first = False
        return first_patches
    # If the search restart has been activated, the search is done in patches for 
    # the horizon, and the left and right sides, the default patches.
    elif restart:
        patches =  [(1.0, (392, 480, 0, w), 1), 
                    (1.3, (392, 497, 0, left + int(83 * 2)), 1), (1.3, (392, 497, right - int(83 * 2), w), 1),
                    #(1.3, (392, 497, 0, w), 1),
                    (1.6, (392, 546, 0, left), 1), (1.6, (392, 546, right, w), 1), 
                    (2.0, (392, 590, 0, left), 1), (2.0, (392, 590, right, w), 1),
                    (2.5, (392, 633, 0, left), 1), (2.5, (392, 633, right, w), 1),
                    (2.85, (392, 666, 0, left), 1), (2.85, (392, 666, right, w), 1)]
    
    # If it is not the first frame or a search restart, the search is restricted 
    # to patches of the cars detected in the previous frame.
    pts_previous_car = []
    if len(car_history) > 0:
        pts_previous_car = car_history[-1]
        
    for pts in pts_previous_car:
        pt1 = pts[0]
        pt2 = pts[1]
        # If the car is within the left margin, the search is restricted to the entire left margin
        if pt2[0] < left : # Car is appearing or disappearing on the left
            new_pt1 = [0, 392]
            xpt2 = left
            if not restart: xpt2 += 1
            new_pt2 = [xpt2, 651]
        # If the car is within the right margin, the search is restricted to the entire right margin
        elif pt1[0] > right: # Car is appearing or disappearing on the right
            xpt1 = right
            if not restart: xpt1 -= 1
            new_pt1 = [xpt1, 392]
            new_pt2 = [w, 651]
        # If the car is not within the margins, a new search patch is created whose size is calculated 
        # on the diagonal of the car's frame, multiplied by a factor.
        else: 
            dist = int(np.linalg.norm(pts)) + 1
            new_dist = int(dist * 1.03)
            aug = (new_dist - dist) // 2
            new_pt1 = np.array(pts[0]) - aug
            new_pt2 = np.array(pts[1]) + aug
            
        # Execute only if no restart or car outside left-right margin.
        if new_pt2[0] > left and new_pt1[0] < right: 
            if new_pt1[1] < 416 and new_pt2[1] > 480: 
                patches.append((1.0, (392, 480, max(new_pt1[0],0), min(new_pt2[0],w)), 1))
            if new_pt1[1] < 416 and new_pt2[1] > 484: 
                patches.append((1.3, (392, 497, max(new_pt1[0],0), min(new_pt2[0],w)), 1))
            if new_pt1[1] < 420 and new_pt2[1] > 503: 
                patches.append((1.6, (392, 546, max(new_pt1[0],0), min(new_pt2[0],w)), 1))
            if new_pt1[1] < 448 and new_pt2[1] > 528: 
                patches.append((2.0, (392, 590, max(new_pt1[0],0), min(new_pt2[0],w)), 1))
            if new_pt2[1] > 560: 
                patches.append((2.5, (392, 633, max(new_pt1[0],0), min(new_pt2[0],w)), 1))
            if new_pt2[1] > 603: 
                patches.append((2.85, (392, 666, max(new_pt1[0],0), min(new_pt2[0],w)), 1))
    
    return patches

### Function to find cars
By means of a sliding window and with the help of the classifier, it determines coordinates that delimit a box of a specific size where there is a vehicle in the image,

In [ ]:
def find_cars(img, shape, color_scale, scale, svc, orient, pix_per_cell, 
              cell_per_block, cells_per_step, all_squares=False):

    # Image conversion to the range values [0,1]
    img = img.astype(np.float32)/255 
    
    # Shape of the new image patch
    ystart, ystop, xstart, xstop = shape
    
    # Crop the image
    img_tosearch = img[ystart:ystop,xstart:xstop,:] 

    # Image conversion to a color scale
    ctrans_tosearch = cv2.cvtColor(img_tosearch, convColor[color_scale])
    
    # Scale of the new image
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), 
                                                       np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    #nfeat_per_block = orient*cell_per_block**2
    
    # Define sampling rate and number of steps along x and y
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire patch
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    boxes = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Make a prediction
            test_prediction = svc.predict(hog_features.reshape(1,-1))
            
            # Create the box
            if test_prediction == 1 or all_squares:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                box = [(xbox_left+xstart, ytop_draw+ystart),(xbox_left+win_draw+xstart,ytop_draw+win_draw+ystart)]
                boxes.append(box)
                
    return boxes

### Function to construct a heatmap of the box list

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

### Function to filter false positive

In [ ]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

### Funtion to draw boxes around detected vehicles

In [ ]:
def draw_labeled_bboxes(img, labels):
    boxes = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        boxes.append(bbox)
    # Return the image
    return img, boxes

### Function with the all steps to detect vehicles in a image

In [ ]:
def pipeline(img):
    global nframes
    global restart
    global first
    if nframes % 15 == 0:
        restart = True
    
    patches = get_image_patches(img, first, restart, car_history)
    restart = False
    first = False

    boxes = []
    for scale, shape, cells_per_step in patches:

        boxes_loop = find_cars(img, shape, color_scale, scale, svc, orient, pix_per_cell, 
                                 cell_per_block, cells_per_step, all_squares=False)
        if len(boxes_loop) > 0:
            boxes.append(boxes_loop)


    flat_list = [item for sublist in boxes for item in sublist]
    
    # If there were cars in the history and they were not detected in the new frame, 
    # the oldest cars in the history are eliminated.
    # And if there are no more cars in the history, the search is restarted.
    if len(car_history) > 0 and len(flat_list) == 0: 
        del car_history[0] 
        if len(car_history) == 0: 
            restart = True
    
    # Add to the list of positive searches with the most recent cars in the history.
    new_list = []
    if len(flat_list) > 0:
        new_list.append(flat_list)
    if len(car_history) > 0 and len(flat_list) < 2:
        new_list.append(np.concatenate(car_history))
    elif len(car_history) > 0:
        new_list.append(car_history[-1])
    
    if len(new_list) > 0:
        new_list = np.concatenate(new_list)

    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    heat = add_heat(heat,new_list)            
        
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    
    # Get image of the drawn boxes and the boxes of the vehicules
    draw_img, boxes = draw_labeled_bboxes(np.copy(img), labels)

    # If any of the cars that appear in the previous frames have been lost, 
    # the search will be restarted.
    if len(car_history) > 0:
        prev_cars = car_history[-1]
        if len(boxes) < len(prev_cars):
            restart = True
             
    # Only one history of three frames is maintained, eliminating the oldest one.
    if len(car_history) == 3: 
        del car_history[0]
        
    # If you have detected cars that are not the additions of the previous frame,
    # they are added to the history.
    if len(boxes) > 0 and len(flat_list) > 1: 
        car_history.append(boxes)
    
    nframes +=1
    return draw_img

### Attributes to control the process of detection
This atributes are:
* A flag for the first frame.
* A counter of the number of frames for the next restart of the search.
* A flag for the search restart.
* A list to hold a history of vehicules from up to three previous frames.

In [ ]:
first = True
nframes = 0
restart = True
car_history = []

In [ ]:
raw_clip = VideoFileClip('project_video.mp4')
clip = raw_clip.fl_image(pipeline)
clip.write_videofile('output_videos/project_video.mp4', audio=False)